In [1]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [2]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math

In [3]:
data = get_input(11, 2017)[0].split(',')

def move(pos, dir):
  x,y = pos
  if dir == 'n':
    return (x, y-2)
  elif dir == 's':
    return (x, y+2)
  elif dir == 'ne':
    return (x+1,y-1)
  elif dir == 'se':
    return (x+1,y+1)
  elif dir == 'sw':
    return (x-1,y+1)
  elif dir == 'nw':
    return (x-1,y-1)

def moves(pos, dirs):
  for dir in dirs:
    pos = move(pos, dir)
  return pos

assert moves((0,0), ['n','s']) == (0,0)
assert moves((0,0), ['ne','sw']) == (0,0)
assert moves((0,0), ['ne','nw', 's']) == (0,0)
assert moves((0,0), ['ne','nw', 's']) == (0,0)
assert moves((0,0), ['se','n', 'nw', 's']) == (0,0)
assert moves((0,0), ['sw','n', 'se']) == (0,0)
assert moves((0,0), ['sw','se', 'n']) == (0,0)

def hex_dist(a,b):
  x,y = a
  x2,y2 = b
  dy = abs(y2-y)
  dx = abs(x2-x)
  steps = 0

  # go diag
  while dx >= 1 and dy >= 1:
    dx -= 1
    dy -= 1
    steps += 1

  if dx == 0 and dy == 0: return steps

  while not (dx == 0 and dy == 0):
    if dx == 0:
      assert dy >= 2
      assert dy % 2 == 0
      steps += dy // 2
      return steps
    if dy == 0:
      steps += dx
      return steps
    assert False
  assert False

pos = moves( (0,0), data)
pos

p1 = hex_dist((0,0), pos)


max_steps = 0
pos = (0,0)
for line in data:
  pos = move(pos, line)
  steps = hex_dist((0,0), pos)
  if steps > max_steps:
    max_steps = steps

p2 = max_steps
p1,p2


(707, 1490)

In [4]:
data = get_input(12, 2017)
G = defaultdict(set)
for line in data:
  src,*targets = aoc_utils.mapints(line)
  G[src] |= set(targets)

def find_group(start):
  seen = set()
  search = [start]
  while len(search):
    cur = search.pop()
    seen.add(cur)
    for t in G[cur]:
      if t in seen: continue
      search.append(t)
  return seen

p1 = len(find_group(0))
p1

groups = []
for src in G:
  if any(src in g for g in groups): continue
  groups.append(find_group(src))
p2 = len(groups)

p1,p2



(113, 202)

In [61]:
list(itertools.chain(range(4), reversed(range(3,1,-1))))
list(range(4)), list(reversed(range(1,3)))
def mk_loop(n):
  from itertools import chain, cycle
  return cycle(chain(range(n), reversed(range(1,n-1))))

list(mk_loop(4))[:8]
# list(itertools.chain(range(1),range(2)))


In [38]:
data = get_input(13, 2017)
testdata ="""0: 3
1: 2
4: 4
6: 4""".split("\n")

from functools import cache

@cache
def calc(rng, time):
  arr = get_rng(rng)
  return arr[time % len(arr)]

@cache
def get_rng(rng):
  from itertools import chain
  arr = list(chain(range(rng), reversed(range(1,rng-1))))
  return arr


def run(delay=0):
  tot = 0
  caught = 0
  for l in data:
    depth,rng = aoc_utils.mapints(l)
    v = calc(rng, depth+delay)
    if v == 0:
      tot += depth*rng
      caught += 1
  return tot,caught

p1,_ = run(delay=0)

p2 = -1
for i in range(3_000_000,10_000_000):
  if run(i)[1] == 0:
    p2 = i
    break

# 34124 is too low
p1,p2

(1612, 3907994)

Counter({'1': 8285, '0': 8099})

In [132]:
def reverse_section(seq, length, pos):
  if length == 31 and pos == 225:
    pass
  orig_len = len(seq)
  end_idx = length + pos
  if end_idx >= len(seq):
    end_idx = end_idx % len(seq)
    prefix = seq[:end_idx % len(seq)]
    suffix = seq[pos:]
    assert len(prefix) + len(suffix) == length
    inner = seq[end_idx:pos]
    assert len(prefix) + len(suffix) + len(inner) == orig_len
    changed = list(reversed(suffix + prefix))
    out_prefix = changed[-len(prefix):] if len(prefix) else []
    out_suffix = changed[:len(suffix)] if len(suffix) else []
    out = out_prefix + inner + out_suffix
  else:
    out = seq[:pos] + list(reversed(seq[pos:pos+length])) + seq[pos+length:]
  assert len(out) == orig_len
  return out
assert reverse_section(list("21034"), 4, 3) == list("43012")
assert reverse_section(list("01234"), 3, 0) == list("21034")
assert reverse_section(list("43012"), 5, 1) == list("34210")

def knot_hash_round(seq, lengths, pos=0, skip=0):
  for length in lengths:
    try:
      seq = reverse_section(seq, length, pos)
    except:
      print(seq,length,pos)
      raise Exception
    pos = (pos + length + skip) % len(seq)
    skip += 1
  return seq,pos,skip

assert knot_hash_round(list(range(5)), [3,4,1,5])[0] == [3,4,2,1,0]

data = mapints(get_input(10, 2017)[0])
lengths = data
out,*_ = knot_hash_round(list(range(256)), lengths)
p1 = out[0]*out[1]
print("p1",p1)

def knot_hash(s):
  from functools import reduce
  from operator import xor

  seq = list(range(256))
  rounds = 64
  pos = 0
  skip = 0
  std_lengths = [17, 31, 73, 47, 23]
  lengths = [ord(ch) for ch in s] + std_lengths
  for _ in range(rounds):
    seq,pos,skip = knot_hash_round(seq, lengths, pos, skip)
  assert len(seq) == 256
  assert len(set(seq)) == len(seq)

  # reduce w/ xor
  out = [ 
    reduce( xor, seq[i*16:(i+1)*16] )
    for i in range(16)
  ]
  assert len(out) == 16
  out = list(map(lambda x: hex(x)[2:].rjust(2, '0'), out))
  return ''.join(out)

assert knot_hash("") == "a2582a3a0e66e6e86e3812dcb672a272"
assert knot_hash("AoC 2017") == "33efeb34ea91902bb2f59c9920caa6cd"
assert knot_hash("1,2,3") == "3efbe78a8d82f29979031a4aa0b16a9d"
assert knot_hash("1,2,4") == "63960835bcdc130f0b66d7ff4f6a5a8e"

p2 = knot_hash(get_input(10, 2017)[0])
assert p2 == "e0387e2ad112b7c2ef344e44885fe4d8"
print("p2",p2)



p1 11375
p2 e0387e2ad112b7c2ef344e44885fe4d8


In [164]:
data = get_input(14, 2017)[0]
from aoc_utils import *
from collections import Counter

# simple way where we don't need to care about the actual positions
def solve1(s):
  C = Counter()
  for i in range(128):
    hash = knot_hash(f"{s}-{i}")
    for ch in hash:
      b = bin(int(ch, 16))[2:].rjust(4, '0')
      C.update(b)
  return C['1']


def solve2(s,is_p2=False):
  G = {}
  for row in range(128):
    hash = knot_hash(f"{s}-{row}")
    for colgroup in range(32):
      v = int(hash[colgroup], 16)
      for offset in range(4):
        col = offset + 4*colgroup
        G[(row,col)] = v & (1 << offset)
  if not is_p2:
    return len([v for v in G.values() if v])
  else:
    allseen = set()
    groups = 0
    for pos in G.keys():
      if not G[pos]: continue
      if pos in allseen: continue

      curgroup = set()
      newgroup = True
      search = [pos]
      while len(search):
        cur = search.pop()
        if not G[cur]: continue
        if cur in allseen: newgroup = False
        if cur in curgroup: continue
        curgroup.add(cur)
        for n in aoc_utils.neighbors4(cur):
          if G.get(n, 0):
            search.append(n)
      if newgroup: groups += 1
      allseen.update(curgroup)
    return groups



assert solve1("flqrgnkx") == solve2("flqrgnkx") == 8108
assert solve1(data) == solve2(data) == 8292
assert solve2(data, True) == 1069

print("p1",solve2(data))
print("p2",solve2(data, True))
